In [1]:
import numpy as np
import matplotlib.pyplot as plt

ph = np.load('Photon.npy')
ele = np.load('Electron.npy')
pi0 = np.load('Pizero.npy')
pipm = np.load('Piplusminus.npy')

In [2]:
data = np.vstack((ph[:100000,:],ele[:100000,:],pi0[:100000,:],pipm[:100000,:]))
datatest = np.vstack((ph[100000:125000,:],ele[100000:125000,:],pi0[100000:125000,:],pipm[100000:125000,:]))
datatest.shape

(100000, 323)

In [3]:
truez = data[:,2]
psb = data[:,20:83]
emb1 = data[:,83:134]
emb2 = data[:,134:197]
emb3 = data[:,197:260]                   
hab1 = data[:,260:]

# sum of all cells
cells_sum = np.sum(data[:,20:],axis=1).reshape(-1,1)

# sum of layers in phi
psb_sum = np.sum(psb.reshape(-1,7,9),axis=1)
emb1_sum = np.sum(emb1.reshape(-1,3,17),axis=1)
emb2_sum = np.sum(emb2.reshape(-1,7,9),axis=1)
emb3_sum = np.sum(emb3.reshape(-1,7,9),axis=1)
hab1_sum = np.sum(hab1.reshape(-1,7,9),axis=1)

# normalised sums
psb_nsum = psb_sum / cells_sum
emb1_nsum = emb1_sum / cells_sum
emb2_nsum = emb2_sum / cells_sum
emb3_nsum = emb3_sum / cells_sum
hab1_nsum = hab1_sum / cells_sum

# center eta of layers
psb_eta = data[:,6]
emb1_eta = data[:,8]
emb2_eta = data[:,10]
emb3_eta = data[:,12]
hab1_eta = data[:,14]
differ1 = psb_eta - emb1_eta
differ2 = emb1_eta - emb2_eta
differ3 = emb2_eta - emb3_eta
differ4 = emb3_eta - hab1_eta

In [4]:
train1 = np.column_stack((psb_nsum[:,3],psb_nsum[:,4],psb_nsum[:,5],emb1_nsum[:,7],emb1_nsum[:,8],emb1_nsum[:,9],
                          emb2_nsum[:,3],emb2_nsum[:,4],emb2_nsum[:,5],emb1_eta,differ1,differ2))

train2 = np.column_stack((psb_nsum,emb1_nsum,emb2_nsum,emb3_nsum,hab1_nsum,
                              psb_eta.reshape(-1,1),emb1_eta.reshape(-1,1),
                          emb2_eta.reshape(-1,1),emb3_eta.reshape(-1,1),hab1_eta.reshape(-1,1),
                              differ1.reshape(-1,1),differ2.reshape(-1,1),differ3.reshape(-1,1),differ4.reshape(-1,1)))

train_target = truez

In [5]:
truezt = datatest[:,2]
psbt = datatest[:,20:83]
emb1t = datatest[:,83:134]
emb2t = datatest[:,134:197]
emb3t = datatest[:,197:260]                   
hab1t = datatest[:,260:]

# sum of all cells
cells_sumt = np.sum(datatest[:,20:],axis=1).reshape(-1,1)

# sum of layers in phi
psb_sumt = np.sum(psbt.reshape(-1,7,9),axis=1)
emb1_sumt = np.sum(emb1t.reshape(-1,3,17),axis=1)
emb2_sumt = np.sum(emb2t.reshape(-1,7,9),axis=1)
emb3_sumt = np.sum(emb3t.reshape(-1,7,9),axis=1)
hab1_sumt = np.sum(hab1t.reshape(-1,7,9),axis=1)

# normalised sums
psb_nsumt = psb_sumt / cells_sumt
emb1_nsumt = emb1_sumt / cells_sumt
emb2_nsumt = emb2_sumt / cells_sumt
emb3_nsumt = emb3_sumt / cells_sumt
hab1_nsumt = hab1_sumt / cells_sumt

# center eta of layers
psb_etat = datatest[:,6]
emb1_etat = datatest[:,8]
emb2_etat = datatest[:,10]
emb3_etat = datatest[:,12]
hab1_etat = datatest[:,14]
differ1t = psb_etat - emb1_etat
differ2t = emb1_etat - emb2_etat
differ3t = emb2_etat - emb3_etat
differ4t = emb3_etat - hab1_etat

In [6]:
test1 = np.column_stack((psb_nsumt[:,3],psb_nsumt[:,4],psb_nsumt[:,5],emb1_nsumt[:,7],emb1_nsumt[:,8],emb1_nsumt[:,9],
                          emb2_nsumt[:,3],emb2_nsumt[:,4],emb2_nsumt[:,5],emb1_etat,differ1t,differ2t))

test2 = np.column_stack((psb_nsumt,emb1_nsumt,emb2_nsumt,emb3_nsumt,hab1_nsumt,
                        psb_etat.reshape(-1,1),emb1_etat.reshape(-1,1),emb2_etat.reshape(-1,1),
                         emb3_etat.reshape(-1,1),hab1_etat.reshape(-1,1),
                        differ1t.reshape(-1,1),differ2t.reshape(-1,1),differ3t.reshape(-1,1),differ4t.reshape(-1,1)))

test_target = truezt

In [7]:
import xgboost as xgb

# create the xgboost model
model1 = xgb.XGBRegressor(
    learning_rate=0.2,    # Controls the step size shrinkage
    n_estimators=600,      # Number of boosting rounds or trees
    max_depth=6,     # Maximum depth of a tree
    objective='reg:squarederror')

model1.fit(train1, train_target)

# create the xgboost model
model2 = xgb.XGBRegressor(
    learning_rate=0.2,    # Controls the step size shrinkage
    n_estimators=600,      # Number of boosting rounds or trees
    max_depth=6,     # Maximum depth of a tree
    objective='reg:squarederror')

model2.fit(train2, train_target)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=600, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [8]:
%timeit ypred1 = model1.predict(test1)

119 ms ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit ypred2 = model2.predict(test2)

128 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
# create the xgboost model
model3 = xgb.XGBRegressor(
    learning_rate=0.2,    # Controls the step size shrinkage
    n_estimators=320,      # Number of boosting rounds or trees
    max_depth=4,     # Maximum depth of a tree
    objective='reg:squarederror')

model3.fit(train1, train_target)

# create the xgboost model
model4 = xgb.XGBRegressor(
    learning_rate=0.2,    # Controls the step size shrinkage
    n_estimators=320,      # Number of boosting rounds or trees
    max_depth=4,     # Maximum depth of a tree
    objective='reg:squarederror')

model4.fit(train2, train_target)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=320, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [11]:
%timeit ypred3 = model3.predict(test1)

38.6 ms ± 2.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%timeit ypred4 = model4.predict(test2)

42.3 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
